In [2]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')

from functions import *
import shap
import xgboost
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from matplotlib import colors as plt_colors

In [3]:
group_main = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/grp/main_grads.csv').iloc[:, 1:]
group_replace = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/grp/replace_grads.csv').iloc[:, 1:]
group_suppress = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/grp/suppress_grads.csv').iloc[:, 1:]
group_clear = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/grp/clear_grads.csv').iloc[:, 1:]

group_main.columns = ['maintain_g1', 'maintain_g2', 'maintain_g3']
group_replace.columns = ['replace_g1', 'replace_g2', 'replace_g3']
group_suppress.columns = ['suppress_g1', 'suppress_g2', 'suppress_g3']
group_clear.columns = ['clear_g1', 'clear_g2', 'clear_g3']

group_grads = pd.concat([group_main, group_replace, group_suppress, group_clear], axis=1).reset_index()

In [4]:
import glob
import pandas as pd
import re

# Define the pattern to match files
pattern = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/sub*_sm_vector/sub*_sm_vector_*_all_ops.csv'

# Use glob to find all files that match the pattern
file_list = sorted(glob.glob(pattern))

# Initialize an empty list to store DataFrames
dataframes = []

for file in file_list:
    # Extract subject ID from the file path
    subject_id = re.search(r'sub(\d+)_', file).group(1)
    
    # Extract the part of the filename before '_all_ops'
    grad = re.search(r'_(g\d+)_all_ops', file).group(1)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file)
    
    # Add the subject ID as a new column
    df['SubID'] = subject_id
    
    # Add the 'grad' as a new column
    df['grad'] = grad
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Combine all DataFrames into one, if necessary
combined_df = pd.concat(dataframes, ignore_index=True)

# Now 'combined_df' contains all data with a 'SubID' and 'grad' column indicating the subject ID and the gradient respectively.
# Adjusting the DataFrame by dropping and reordering columns as required
combined_df = combined_df.drop(['Unnamed: 0', 'sub'], axis=1, errors='ignore')  # errors='ignore' handles cases where these columns might not exist
combined_df = combined_df[['index', 'SubID', 'maintain', 'replace', 'suppress', 'clear', 'grad']]

# combined_df now includes the 'grad' column with the part of the filename you were interested in.

combined_df_g1 = combined_df.query('grad == "g1"').drop('grad', axis=1).reset_index(drop=True)
combined_df_g1.columns = ['index', 'SubID', 'maintain_g1', 'replace_g1', 'suppress_g1', 'clear_g1']

combined_df_g2 = combined_df.query('grad == "g2"').drop('grad', axis=1).reset_index(drop=True)
combined_df_g2.columns = ['index', 'SubID', 'maintain_g2', 'replace_g2', 'suppress_g2', 'clear_g2']

combined_df_g3 = combined_df.query('grad == "g3"').drop('grad', axis=1).reset_index(drop=True)
combined_df_g3.columns = ['index', 'SubID', 'maintain_g3', 'replace_g3', 'suppress_g3', 'clear_g3']

combined_all = pd.concat([combined_df_g1, 
                          combined_df_g2.drop(['index', 'SubID'], axis=1), 
                          combined_df_g3.drop(['index', 'SubID'], axis=1)], axis=1)

combined_all = combined_all[['index', 'SubID', 'maintain_g1', 'maintain_g2', 'maintain_g3', 'replace_g1', 'replace_g2', 'replace_g3', 
              'suppress_g1', 'suppress_g2', 'suppress_g3', 'clear_g1', 'clear_g2', 'clear_g3']]

wm_networks = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/rest/rest_gradients/wm_networks.csv')
wm_networks = wm_networks.reset_index()

combined_all = pd.merge(wm_networks, combined_all, on='index').sort_values(['SubID', 'index'])

group_grads = pd.merge(wm_networks, group_grads, on='index')#.sort_values(['SubID', 'index'])

In [ ]:
combined_all

In [ ]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')

from functions import *
import shap
import xgboost
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from matplotlib import colors as plt_colors

classes = ['1', '2', '3', '4']

class_names = list(group_grads.Subtype.unique())

def classy(sub, op):
    
    global group_grads, combined_all
    
    X_train = group_grads.filter(regex=op)
    X_test = group_grads[['Subtype']]
    y_train = combined_all.query('SubID == @sub').filter(regex=op)
    y_test = combined_all.query('SubID == @sub')[['Subtype']]

    from scipy import stats
    xcols = X_train.columns.to_list()
    #X_train = pd.DataFrame(stats.zscore(X_train), columns = xcols)
    #y_train = pd.DataFrame(stats.zscore(y_train), columns = xcols)

    cls = xgboost.XGBClassifier(objective='multi:softmax')

    cls.fit(X_train, X_test)

    predictions = cls.predict(y_train)
    report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True)).iloc[:3, :]
    report.columns = ['VN', 'SMN', 'FPCN', 'DMN', 'accuracy', 'macro avg', 'weighted avg']
    report = report.assign(SubID = sub)
    
    return report

In [ ]:
maintain_reports = []
for i in combined_all.SubID.unique():
    maintain_reports.append(classy(i, 'maintain'))
    
replace_reports = []
for i in combined_all.SubID.unique():
    replace_reports.append(classy(i, 'replace'))
    
suppress_reports = []
for i in combined_all.SubID.unique():
    suppress_reports.append(classy(i, 'suppress'))
    
clear_reports = []
for i in combined_all.SubID.unique():
    clear_reports.append(classy(i, 'clear'))
    
reports = []
for i in combined_all.SubID.unique():
    reports.append(classy(i, 'g'))

In [ ]:
def clean_reports(report_list, op=None):
    
    report_df = pd.concat(report_list).reset_index().sort_values(['index', 'SubID'])
    report_df = (report_df[['SubID', 'index', 'VN', 'SMN', 'FPCN', 'DMN', 'accuracy']]
                 .rename({'index':'metrics'}, axis=1))

    report_f1 = report_df.query('metrics == "f1-score"').drop(['metrics', 'accuracy'], axis=1).reset_index(drop=True)
    report_f1.columns = ['SubID', 'VN_f1', 'SMN_f1', 'FPCN_f1', 'DMN_f1']

    report_precision = report_df.query('metrics == "precision"').drop(['metrics', 'accuracy'], axis=1).reset_index(drop=True)
    report_precision.columns = ['SubID', 'VN_precision', 'SMN_precision', 'FPCN_precision', 'DMN_precision']

    report_recall = report_df.query('metrics == "f1-score"').drop(['metrics', 'accuracy'], axis=1).reset_index(drop=True)
    report_recall.columns = ['SubID', 'VN_recall', 'SMN_recall', 'FPCN_recall', 'DMN_recall']

    report_accuracy = report_df[['SubID', 'accuracy']].drop_duplicates().reset_index(drop=True)

    report_final = pd.merge(report_f1, report_precision, on ='SubID')
    #report_final = pd.merge(report_recall, report_final, on ='SubID')
    report_final = pd.merge(report_accuracy, report_final, on ='SubID')

    report_final[['SubID']] = report_final[['SubID']].astype(int)
    
    if op is not None:
        report_final.columns = ['SubID'] + [col + '_'+op for col in report_final.iloc[:, 1:].columns.to_list()]

    return report_final

In [ ]:
maintain_report = clean_reports(maintain_reports, 'maintain')
replace_report = clean_reports(maintain_reports, 'replace')
suppress_report = clean_reports(maintain_reports, 'suppress')
clear_report = clean_reports(maintain_reports, 'clear')

report_df = clean_reports(reports)

In [ ]:
all_report_df = pd.merge(maintain_report, replace_report, on ='SubID')
all_report_df = pd.merge(all_report_df, suppress_report, on ='SubID')
all_report_df = pd.merge(all_report_df, clear_report, on ='SubID')

In [ ]:
matched_subid = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/matched_subid.csv')
matched_subid.columns = ['sub', 'SubID']

In [ ]:
report_final = pd.merge(matched_subid, report_df, on='SubID').drop('SubID', axis=1).rename({'sub':'SubID'}, axis=1)

all_report_final = pd.merge(matched_subid, all_report_df, on='SubID').drop('SubID', axis=1).rename({'sub':'SubID'}, axis=1)

In [ ]:
z_data = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/ClearMem_Z_Average.csv')
z_data = z_data[['SubID', 'z_ave', 'PSWQ_total', 'WBSI_total', 'RRS_total', 'RRS_depression', 'RRS_brooding', 'RRS_reflection']]
z_data = z_data.dropna()

In [ ]:
report_z = pd.merge(z_data, report_final, on='SubID')
all_report_z = pd.merge(z_data, all_report_final, on='SubID')

report_z = report_z.query('SubID != 76')
all_report_z = all_report_z.query('SubID != 76')

In [ ]:
def pls(data, target, networks, color, save=None):


    regex_string = '|'.join(networks)

    import seaborn as sns
    from sklearn.cross_decomposition import PLSRegression
    from scipy.stats import t
    import numpy as np
    import matplotlib.pyplot as plt
    import statsmodels.api as sm

    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import RobustScaler
    from sklearn.preprocessing import StandardScaler
    
    from sklearn.model_selection import cross_val_predict
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import LeaveOneOut
    # Create an instance of LeaveOneOut
    loo = LeaveOneOut()

    
    scaler=MinMaxScaler()
    
    X_data = data
    # Extract the feature matrix X and target variable y
    X = (X_data 
             .filter(regex=regex_string)
             #.filter(like='dispersion_md')
             #.filter(regex='^(?!.*(main|suppress|replace)).*$')
             #.filter(regex='^(?!.*(maintain_smn)).*$') 
            )
    #print(X.columns)

    # fit and transform the entire dataframe
    #X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    y = X_data[target]
    y = pd.DataFrame(scaler.fit_transform(y), columns=y.columns)

    # Set the number of permutations and bootstrap repetitions
    n_permutations = 10000
    n_bootstrap = 10000

    pls = PLSRegression(n_components=1)
        
    # Perform leave-one-out cross-validation to determine the optimal number of components
    #mse_values = []
    #for n_components in range(1, len(X.columns)+1):
    #    pls.n_components = n_components
    #    y_pred = cross_val_predict(pls, X, y, cv=loo)
    #    mse = mean_squared_error(y, y_pred)
    #    mse_values.append(mse)

    #pls = PLSRegression(n_components=1)
    #optimal_n_components = np.argmin(mse_values) + 1
    #print('Optimal number of components:', optimal_n_components)

    # Fit the PLSRegression model to the feature matrix X and target variable y using the optimal number of components
    pls.n_components = 1 #optimal_n_components
    pls.fit(X, y)

    # Compute the singular value of the fitted model
    x_score = pls.x_scores_
    y_score = pls.y_scores_
    singular_value = np.dot(x_score.T, y_score)
    print('Starting Permutations')
    # Generate permuted data matrices by randomly reordering the rows of the original data matrix X
    permuted_singular_values = []
    for i in range(n_permutations):
        permuted_X = X.sample(frac=1, replace=False)
        pls.fit(permuted_X, y)
        x_score = pls.x_scores_
        y_score = pls.y_scores_
        permuted_singular_value = np.dot(x_score.T, y_score)
        permuted_singular_values.append(permuted_singular_value)

    # Compute the p-value of the singular value by comparing it to the null distribution of singular values
    p_value = (np.sum(permuted_singular_values >= singular_value) + 1) / (n_permutations + 1)
    
    import seaborn as sns
    from sklearn.cross_decomposition import PLSRegression
    from scipy.stats import t
    import numpy as np
    import matplotlib.pyplot as plt
    import statsmodels.api as sm
    import matplotlib

       # Prepare data for the histogram
    singular_list = [i[0][0] for i in permuted_singular_values]
    singulars = pd.DataFrame(singular_list, columns=['Singulars'])
    singular_val = singular_value[0][0]

    # Begin plotting
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))  # Set up a figure with two subplots
    
    # Scatter plot on the left
    sns.set_theme(style="white")

    sns.regplot(x=x_score.flatten(), y=y_score.flatten(), 
                ax=axs[0], color=color, scatter_kws={'alpha':0.6}, truncate=False)
    axs[0].set_xlabel('Network Dispersion (Latent Variable)', fontsize=14)
    axs[0].set_ylabel('Behavior (Latent Variable)', fontsize=14)
    axs[0].set_title('Network Dispersion Predicting Behavior', fontsize=14, fontweight='bold')
    
       # Displaying 'permuted P = ' with manual positioning for readability
    axs[0].text(.06, 1, 'permuted ', transform=axs[0].transAxes, fontsize=13, verticalalignment='top', color='black')
    axs[0].text(.22, 1, 'P', transform=axs[0].transAxes, fontsize=13, verticalalignment='top', color='black', fontstyle='italic')
    axs[0].text(.24, 1, ' = {:.3f}'.format(p_value), transform=axs[0].transAxes, fontsize=13, verticalalignment='top', color='black')
    
    sns.despine(ax=axs[0])

    # Histogram on the right
    sns.histplot(singulars, kde=False, ax=axs[1])
    axs[1].axvline(x=singular_val, color='r', linestyle='--', label='Singular Value')
    axs[1].legend()
    axs[1].set_title("Singular Value vs. Null Distribution", fontsize=14, fontweight='bold')
    axs[1].set_xlabel('Permuted Singular Values (Null)', fontsize=14)
    sns.despine(ax=axs[1])
    plt.tight_layout()  # Adjust layout to not overlap
    
    if save is not None:
        save_path = f"/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/pls_plots/{save}.png"
        plt.savefig(save_path, dpi=700)
        
    plt.tight_layout()  # Adjust layout to not overlap
    plt.show()
    

    return p_value, x_score, y_score, permuted_singular_values, singular_value


In [ ]:
targets = [['RRS_total', 'PSWQ_total', 'WBSI_total'], ['z_ave']]
colors = ['#64b3a3', '#b19cd9']
networks = ['accuracy','VN', 'SMN', 'FPCN', 'DMN']
names = ['rrs_pws_wbsi_classacc', 'z_ave_classacc']
for i,j,k in zip(targets, colors, names):
    p_value, x_score, y_score, permuted_singular_values, singular_value = pls(report_z, i, networks, j, k)

In [ ]:
targets = [['RRS_total', 'PSWQ_total', 'WBSI_total'], ['z_ave']]
colors = ['#64b3a3', '#b19cd9']
networks = ['accuracy','VN', 'SMN', 'FPCN', 'DMN']

names = ['rrs_pws_wbsi_allops_classacc', 'z_ave_allops_classacc']
for i,j,k in zip(targets, colors, names):
    p_value, x_score, y_score, permuted_singular_values, singular_value = pls(all_report_z, i, networks, j, k)